### Install and import the necessary python packages
- `praw` an acronym for "Python Reddit API Wrapper", is a [Python package](https://github.com/praw-dev/praw) that allows for simple access to Reddit's API. We need this to fetch data from reddit.
- `pandas` is a datascience python package and we need this to save the data to `csv` file.

However, we need to install these packages to our virtual environment.
The following code will install them. If you have installed them already, you may skip running it or comment the follwoing cell.

In [ ]:
!pip install praw
!pip install pandas

We import the necessary packages

In [ ]:
import praw
import pandas as pd
import datetime as dt

Now, we need to connect to reddit using the api information from our reddit account. If you do not have it, you need to create it within your account. To do so:
- Login to your reddit account in a browswer.
  - It is better to use the username and password to create your reddit account, since you need it later on.
    - If you have already created your account using google, apple or any other authenticators, you need to create a password for your account. You may try to delete your account and create a new one. However, during the delete process, it will ask you to assing a password to your account (a hack :smiley:)
- You need to create an App witin your reddit account. To do so:
  - After login, open this following page: [https://www.reddit.com/prefs/apps/](https://www.reddit.com/prefs/apps/)
  - Click on `create another app...`
  - Give it a name i.e. `api-test`
  - **Choose `script`**
  - You may write a description which is optional.
  - "about url" is optional.
  - For `redirect url` you may use `https://www.google.com/`

<img src=./Assets/images/create-reddit-app.png>

  - Then the app would be like this:

<img src=./Assets/images/created-reddit-app.png>

To connect to your app using `praw`, in the following code, use:
- `client_id` is written after "personal use script"
- `client_secret` is written in front of the "secret"
- `username` can be your username or the one written in front of the "developers"
- `password` you know your reddit password
- `user_agent` can be anything i.e. "api-test"


In [ ]:
reddit = praw.Reddit(client_id = '<The client_id or the personal use script>',
                     client_secret = '<The secret variable in your reddit app>',
                     username='<your user name or the developers variable>',
                     password='<your password>',
                     user_agent='<This can be any phrase or the name of your api>')

Now, you may choose your topic that you want to fetch data regarding it from reddit. For example you may choose "Covid19".
Then you need to create a `subreddit` variable to read the information from it.

In [ ]:
topic_name = "Covid19"
subreddit = reddit.subreddit(topic_name)

You may choose which variables from the reddit information you want to retriev. You may choose the followings, and create a `dict` to save them from each reddit record.

To fetrch the data, you may choose to get at most 100 "hot" pages based on your topic using `hot` function and save them in a pandas dataform.

In [ ]:
topics_dict = { "title":[], \
                "score":[], \
                "id":[], "url":[], \
                "comms_num": [], \
                "created": [], \
                "post_comment":[], \
                "upvotes":[]}
for submission in subreddit.hot(limit=100):
    topics_dict["title"].append(submission.title)
    topics_dict["score"].append(submission.score)
    topics_dict["id"].append(submission.id)
    topics_dict["url"].append(submission.url)
    topics_dict["comms_num"].append(submission.num_comments)
    topics_dict["created"].append(submission.created)
    topics_dict["post_comment"].append(submission.selftext)
    topics_dict["upvotes"].append(submission.ups)
    
    topics_data = pd.DataFrame(topics_dict)

If you recieve any error similar to
```
OAuthException: invalid_grant error processing request
```
you might have misspelled any of the client_id, client_secret, username or password.



Now, you have fetched all the data, but you need to save them somewhere i.e. a csv file.
To have better view of the date and time of the created reddit page, you may change the `timestamp` of it to the human readable date using the defined `get_date` function and add it to the last column of your fetched data.
Fianlly, you may save it in a csv file.

In [ ]:
def get_date(created):
    return dt.datetime.fromtimestamp(created)

_timestamp = topics_data["created"].apply(get_date)
topics_data = topics_data.assign(timestamp = _timestamp)

topics_data.to_csv(f'{topic_name}_topics.csv') 